In [17]:
import numpy as np
import pandas as pd
import numpy as np

def normalise(a):
    if a<0:
        a=-a
    a=a/np.power(10,np.floor(np.log10(a)))
    return a

    

In [18]:
df = pd.read_csv('train.csv', low_memory=False)
test = pd.read_csv('test.csv', low_memory=False)
test.drop(columns=['Id'],inplace=True)
df = pd.concat([df,test])

df['Location'] = df['Location'].str.replace(",",' ')
# df[['loc1','loc2']] = df.Location.str.split(' ',expand = True).astype(float)
# print(df['loc2'].dtype)
# print(len(df[]))
# for i in range(len(df.index)):
# print(df['loc2'].shape)

df['Crash Date/Time'] = pd.to_datetime(df['Crash Date/Time'])
df['Hour'] = df['Crash Date/Time'].dt.hour
df['day'] = df['Crash Date/Time'].dt.dayofweek
df['Agency Name'] = df['Agency Name'].str.replace("Montgomery County Police",'MONTGOMERY')
df['Driver Substance Abuse'] = df['Driver Substance Abuse'].str.replace("CONTRIBUTED",'PRESENT')
df['Driver Substance Abuse'] = df['Driver Substance Abuse'].str.replace("COMBINED SUBSTANCE",'COMBINATION')
# df['ACRS Report Type'].fillna("UNKNOWN", inplace=True)
df.drop(columns = ['Report Number','Local Case Number','Road Name','Cross-Street Name','Off-Road Description','Municipality','Location','Circumstance','Vehicle Model', 'Vehicle Make','Vehicle ID','Road Name','Person ID','Driverless Vehicle','Crash Date/Time','Vehicle Year'],inplace=True)

# df['Route Type'].fillna("UNKNOWN", inplace=True)
# df['Cross-Street Type'].fillna("UNKNOWN", inplace=True)
# df['Collision Type'].fillna("UNKNOWN", inplace=True)
# df['Parked Vehicle'].fillna("UNKNOWN", inplace=True)
# df['Vehicle First Impact Location'].fillna("UNKNOWN", inplace=True)
# df['Vehicle Second Impact Location'].fillna("UNKNOWN", inplace=True)
# df['Vehicle Damage Extent'].fillna("UNKNOWN", inplace=True)
# df['Injury Severity'].fillna("UNKNOWN", inplace=True)
# df['Vehicle Movement'].fillna("UNKNOWN", inplace=True)
# df['Driver Substance Abuse'].fillna("UNKNOWN", inplace=True)
# df['Non-Motorist Substance Abuse'].fillna("UNKNOWN", inplace=True)
# df['Weather'].fillna("CLEAR", inplace=True)
# df['Surface Condition'].fillna("DRY", inplace=True)
# df['Light'].fillna("DAYLIGHT", inplace=True)
# df['Vehicle Body Type'].fillna("PASSENGER CAR", inplace=True)
# df['Traffic Control'].fillna("UNKNOWN", inplace=True)
# df['Agency Name'].fillna("UNKNOWN", inplace=True)
# df['Vehicle Continuing Dir'].fillna("UNKNOWN", inplace=True)
# df['Vehicle Going Dir'].fillna("UNKNOWN", inplace=True)
# df['Equipment Problems'].fillna("NO MISUSE", inplace=True)

one_hot = pd.get_dummies(df)
m = one_hot.shape[1]
# one_hot.info()
# one_hot.head()
# one_hot['Agency Name_MONTGOMERY'].value_counts(dropna=False)



In [19]:
print(m)
dr = np.random.randint(m,size=5)
# one_hot.drop(one_hot.columns[dr] ,axis=1, inplace=True)
one_hot.info()

325
<class 'pandas.core.frame.DataFrame'>
Int64Index: 128725 entries, 0 to 77234
Columns: 325 entries, Speed Limit to Equipment Problems_UNKNOWN
dtypes: float64(3), int64(3), uint8(319)
memory usage: 46.0 MB


In [20]:

label = one_hot['Fault'].to_numpy(dtype=float)
train = one_hot.drop(columns = ['Fault']).to_numpy(dtype=float)
train.shape
# print('hello',df['loc1'][0],df['loc2'][1])#=normalise(df['loc2'][0])
# 

(128725, 324)

In [21]:
print(train.shape)
test = train[51490:]
train = train[:51490]
label = label[:51490]
from sklearn.model_selection import train_test_split

x,xt,y,yt = train_test_split(train, label, test_size=0.2)
test.shape

(128725, 324)


(77235, 324)

In [22]:
# from sklearn.ensemble import GradientBoostingClassifier
# from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

clf = LGBMClassifier(boosting_type='dart', drop_rate=0.5, max_drop=50, n_iter=1000)
clf.fit(train , label)
# pred = clf.predict(xt)
# print(yt.shape, pred.shape)
# print(accuracy(yt, pred))
# print(yt, pred)

[LightGBM] [Warning] num_iterations is set=1000, n_iter=1000 will be ignored. Current value: num_iterations=1000


/home/cmaspi/.local/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LGBMClassifier(boosting_type='dart', drop_rate=0.5, max_drop=50, n_iter=1000)

In [23]:
clf.score(xt, yt)


0.8923091862497572

In [24]:
pred = clf.predict(test)
n = len(pred)
pred =np.c_[np.linspace(0,n-1,n),pred]
pred=np.array(pred, dtype=int)
output = pd.DataFrame(pred)
output.to_csv('output.csv', header= ['Id','Fault'],index=False)
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77235 entries, 0 to 77234
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       77235 non-null  int64
 1   1       77235 non-null  int64
dtypes: int64(2)
memory usage: 1.2 MB
